In [2]:
import pandas as pd
import numpy as np
import seaborn as sea
import datetime as dt
import matplotlib.pyplot as pp
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [6]:
#converter para o aprendizado supervisionado
def series_to_super(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    dataf = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, 1):
        cols.append(dataf.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(dataf.shift(-1))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
        egg = pd.concat(cols, axis=1)
        egg.columns = names
        if dropnan:
            egg.dropna(inplace=True)
        return egg  
dataset = pd.read_csv('in.csv', header = 0, index_col = 0)
values = dataset.values
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
values = values.astype('float32')
scaler = MinMaxScaler(feature_range = (0,1))
scaled =scaler.fit_transform(values)
n_values = 33207
n_features = 24
reframed = series_to_super(scaled,n_values,n_features)
#reframed.drop(reframed.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]], axis=1, inplace=True)
values = reframed.values
n_train = dataset.size
train = values[:n_train, :]
test = values[n_train:, :]
n_obs = n_values * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
train_X = train_X.reshape((train_X.shape[0], n_features))
test_X = test_X.reshape((test_X.shape[0],  n_features))
#
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae',opitmizer='adam')
#history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
#pp.plot(history.history['loss'], label='train')
#pp.plot(history.history['val_loss'], label='test')
#pp.legend()
#pp.show()

#yhat = model.predict(test_X)
#test_X = test_X.reshape((test_X.shape[0], n_days*n_features))
#inv_yhat = concatenate((yhat, test_X[:, -7:]), axis=1)
#inv_yhat = scaler.inverse_transform(inv_yhat)
#inv_yhat = inv_yhat[:,0]
#test_y = test_y.reshape((len(test_y), 1))
#inv_y = concatenate((test_y, test_X[:, -7:]), axis=1)
#inv_y = scaler.inverse_transform(inv_y)
#inv_y = inv_y[:,0]
#
#rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
#print('Test RMSE: %.3f' % rmse)

IndexError: tuple index out of range

In [4]:
values.shape

(33207, 24)